The EDA and Backward elimination has helped us find the optimal columns for regression training
Now we need to find out what results the machine learning algorithm will give
Here we try XGBoost.

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
import matplotlib.pyplot as plt
import xgboost as xg
from pathlib import Path
from openFile import OpenCleanFile

In [5]:
clean = OpenCleanFile()
clean.df_housing.head()
selected_columns = ["COMMERCIAL UNITS", "TOTAL UNITS", "AGE", "TAX CLASS AT PRESENT", "TAX CLASS AT TIME OF SALE", "BOROUGH_NAME"]  
X, y = clean.df_housing[selected_columns], clean.df_housing["SALE PRICE"]
X.head()  


,COMMERCIAL UNITS,TOTAL UNITS,AGE,TAX CLASS AT PRESENT,TAX CLASS AT TIME OF SALE,BOROUGH_NAME
0,0,5,122,2A,2,Manhattan
1,0,10,109,2B,2,Manhattan
2,0,6,122,2A,2,Manhattan
3,0,8,102,2B,2,Manhattan
4,0,24,102,2,2,Manhattan


Turn the category columns into one-hot encoded columns per category and then select from those columns
the ones that were calculated from the backward elimination process

In [7]:
X = pd.get_dummies(X, drop_first=True)
selected_columns_all = ["COMMERCIAL UNITS", "TOTAL UNITS", "AGE", 
"TAX CLASS AT PRESENT_2", "TAX CLASS AT PRESENT_2A", "TAX CLASS AT PRESENT_2B", 
"TAX CLASS AT TIME OF SALE_2", "TAX CLASS AT TIME OF SALE_4", 
"BOROUGH_NAME_Brooklyn", "BOROUGH_NAME_Queens"]

X = X[selected_columns_all]
X.head()

,COMMERCIAL UNITS,TOTAL UNITS,AGE,TAX CLASS AT PRESENT_2,TAX CLASS AT PRESENT_2A,TAX CLASS AT PRESENT_2B,TAX CLASS AT TIME OF SALE_2,TAX CLASS AT TIME OF SALE_4,BOROUGH_NAME_Brooklyn,BOROUGH_NAME_Queens
0,0,5,122,0,1,0,1,0,0,0
1,0,10,109,0,0,1,1,0,0,0
2,0,6,122,0,1,0,1,0,0,0
3,0,8,102,0,0,1,1,0,0,0
4,0,24,102,1,0,0,1,0,0,0


Now run the model

In [10]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42)
# create an xgboost regression model
model = xg.XGBRegressor()
# Instantiation
xgb_r = xg.XGBRegressor(objective ='reg:squarederror',
                  n_estimators = 10, seed = 123)
 
# Fitting the model
xgb_r.fit(train_X, train_y)
 
# Predict the model
pred = xgb_r.predict(test_X)
 
# RMSE Computation
rmse = np.sqrt(MSE(test_y, pred))
print("RMSE : % f" %(rmse))

RMSE :  13755976.838698
